# UFC Fight Card Predictor

## About this Project:

### Project Goal: 

The goal of this project is to be able to read in an upcoming fight card and predict the outcome of each fight. 


For this particular project we will be looking at UFC 272 PPV - Colby Covington vs. Jorge Masvidal.

### Project Description: 

## Let's Get Started... 

### Imports

In [1]:
import numpy as np
import pandas as pd

# Visualizing
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:.2f}'.format

# Split 
from sklearn.model_selection import train_test_split

# Scale
from sklearn.preprocessing import MinMaxScaler

# Stats
import scipy.stats as stats

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

### Acquire

The first thing we have to do is acquire the data. I will be using a dataset called ufc-master.csv that I found here: 

https://www.kaggle.com/bloodprashure/ufc-p4p-1-dataset/tasks?taskId=4669

In [2]:
# read csv locally 
ufc = pd.read_csv('ufc-master-cleaned.csv')

In [3]:
# verify data was properly acquired
ufc.head(1)

,Unnamed: 0,date,event_code,event_name,fullname,fighter_two_name,w,l,d,nc,...,round_five_takedown_percent,round_five_submission_attempt,round_five_submission_reverse,round_five_control_time,round_five_head,round_five_body,round_five_leg,round_five_distance,round_five_clinch,round_five_ground
0,0,2021-03-06 0:00:00,6e2b1d631832921d,UFC 259: Blachowicz vs. Adesanya,Aalon Cruz,Uros Medic,0,1,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# check row/column shape
ufc.shape

(12156, 171)

### Prepare

In [11]:
# Drop duplicates

print(ufc.shape)
df = ufc.drop_duplicates()
print(ufc.shape)

(12156, 171)
(12156, 171)


In [17]:
# check datatypes
ufc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12156 entries, 0 to 12155
Columns: 171 entries, Unnamed: 0 to round_five_ground
dtypes: float64(10), int64(91), object(70)
memory usage: 15.9+ MB


In [9]:
ufc.isnull().sum()

Unnamed: 0                 0
date                       0
event_code                 0
event_name                 0
fullname                   0
                       ...  
round_five_body        11752
round_five_leg         11752
round_five_distance    11752
round_five_clinch      11752
round_five_ground      11752
Length: 171, dtype: int64

In [7]:
ufc.head()

,Unnamed: 0,date,event_code,event_name,fullname,fighter_two_name,w,l,d,nc,...,round_four_knockdown,round_four_significant_strikes_percent,round_four_takedown_percent,round_four_submission_attempt,round_four_submission_reverse,round_five_knockdown,round_five_significant_strikes_percent,round_five_takedown_percent,round_five_submission_attempt,round_five_submission_reverse
0,0,2021-03-06 0:00:00,6e2b1d631832921d,UFC 259: Blachowicz vs. Adesanya,Aalon Cruz,Uros Medic,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2020-02-29 0:00:00,fc9a9559a05f2704,UFC Fight Night: Benavidez vs. Figueiredo,Aalon Cruz,Spike Carlyle,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2000-11-17 0:00:00,da6dfd09cca1d705,UFC 28: High Stakes,Aaron Brink,Andrei Arlovski,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,2014-08-23 0:00:00,063649e21bc9d6d5,UFC Fight Night: Henderson vs Dos Anjos,Aaron Phillips,Matt Hobar,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,2020-07-15 0:00:00,18f5669a92e99d92,UFC Fight Night: Kattar vs. Ige,Aaron Phillips,Jack Shore,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
ufc.shape

(12156, 143)

### Explore

In [22]:
c_cov = ufc[ufc.fighter_one_name.isin(['Colby Covington'])]

In [23]:
c_cov.head()

,Unnamed: 0,Date,Event-Code,Event-Name,fighter_one_name,fighter_two_name,W,L,D,NC,...,Round-5-Takedown-Percent,Round-5-Submission-Attempt,Round-5-Submission-Reverse,Round-5-Control-Time,Round-5-Head,Round-5-Body,Round-5-Leg,Round-5-Distance,Round-5-Clinch,Round-5-Ground
2211,2211,2019-08-03 00:00:00,03da33a102d9a45f,UFC Fight Night: Covington vs. Lawler,Colby Covington,Robbie Lawler,1,0,0,0,...,0,0,0,0:09,30 of 121,7 of 9,4 of 4,38 of 129,3 of 5,0 of 0
2212,2212,2018-06-09 00:00:00,2eae41f61776c60f,UFC 225: Whittaker vs. Romero 2,Colby Covington,Rafael Dos Anjos,1,0,0,0,...,33,0,0,2:49,13 of 46,3 of 6,4 of 4,19 of 53,1 of 3,0 of 0
2213,2213,2016-12-17 00:00:00,32541eb5d12668b4,UFC on FOX: VanZant vs. Waterson,Colby Covington,Bryan Barberena,1,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2214,2214,2017-06-17 00:00:00,353de740bb6c7e75,UFC Fight Night: Holm vs. Correia,Colby Covington,Dong Hyun Kim,1,0,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2215,2215,2015-12-12 00:00:00,354808cf38d9d73c,UFC 194: Aldo vs McGregor,Colby Covington,Warlley Alves,0,1,0,0,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
